In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("processed.csv")
df

,Unnamed: 0,clean_comment,category
0,0,family mormon never tried explain still stare ...,1
1,1,buddhism much lot compatible christianity espe...,1
2,2,seriously say thing first get complex explain ...,-1
3,3,learned want teach different focus goal not wr...,0
4,4,benefit may want read living buddha living chr...,1
...,...,...,...
36241,37243,agree push make nation either pity pakistan in...,-1
36242,37244,jesus,0
36243,37245,kya bhai pure saal chutiya banaya modi aur jab...,1
36244,37246,downvote karna tha par upvote hogaya,0


In [3]:
df.dropna(inplace=True)

In [4]:
import dagshub
dagshub.init(repo_owner='rajatchauhan99', repo_name='Youtube-Comment-Analysis-Chrome-Plugin', mlflow=True)

Accessing as rajatchauhan99

Initialized MLflow to track repo "rajatchauhan99/Youtube-Comment-Analysis-Chrome-Plugin"

Repository rajatchauhan99/Youtube-Comment-Analysis-Chrome-Plugin initialized!

In [5]:
# Set or create an experiment
import mlflow
mlflow.set_experiment("Exp 5 - Trying Different ML Algos with HP Tuning")

2024/10/19 18:30:17 INFO mlflow.tracking.fluent: Experiment with name 'Exp 5 - Trying Different ML Algos with HP Tuning' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/96c047f2f48c4d168382939725e408d8', creation_time=1729342817379, experiment_id='5', last_update_time=1729342817379, lifecycle_stage='active', name='Exp 5 - Trying Different ML Algos with HP Tuning', tags={}>

In [6]:
import optuna
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

c:\Users\rajat.chauhan\conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# Step 1: Remap the class labels from [-1, 0, 1] to [2, 0, 1]
df['category'] = df['category'].map({-1: 2, 0: 0, 1: 1})

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

# Step 3: TF-IDF vectorizer setup
ngram_range = (1, 3)  # Trigram
max_features = 1000  # Set max_features to 1000
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X = vectorizer.fit_transform(df['clean_comment'])
y = df['category']

# Step 4: Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


# Step 6: Optuna objective function for LightGBM
def objective_lightgbm(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 10)

    model = LGBMClassifier(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth, random_state=42)
    return accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test))


# Step 7: Run Optuna for LightGBM, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_lightgbm, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = LGBMClassifier(n_estimators=best_params['n_estimators'], learning_rate=best_params['learning_rate'], max_depth=best_params['max_depth'], random_state=42)

    # Log the best model with MLflow, passing the algo_name as "LightGBM"
    log_mlflow("LightGBM", best_model, X_train, X_test, y_train, y_test)

# Run the experiment for LightGBM
run_optuna_experiment()

[I 2024-10-19 18:30:27,909] A new study created in memory with name: no-name-da1f89dd-abb5-496a-8f92-87e8f0ff9065


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.033596 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 99989
[LightGBM] [Info] Number of data points in the train set: 37598, number of used features: 977
[LightGBM] [Info] Start training from score -1.098586
[LightGBM] [Info] Start training from score -1.098586
[LightGBM] [Info] Start training from score -1.098665
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further s

[I 2024-10-19 18:30:32,404] Trial 0 finished with value: 0.6935106382978723 and parameters: {'n_estimators': 225, 'learning_rate': 0.01693385460755084, 'max_depth': 5}. Best is trial 0 with value: 0.6935106382978723.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Auto-ch

[I 2024-10-19 18:30:33,900] Trial 1 finished with value: 0.6103191489361702 and parameters: {'n_estimators': 56, 'learning_rate': 0.01231695881156698, 'max_depth': 5}. Best is trial 0 with value: 0.6935106382978723.


...

[I 2024-10-19 18:46:29,043] Trial 28 finished with value: 0.622872340425532 and parameters: {'n_estimators': 227, 'learning_rate': 0.00010078439683929927, 'max_depth': 10}. Best is trial 26 with value: 0.8059574468085107.


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.163122 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 99989
[LightGBM] [Info] Number of data points in the train set: 37598, number of used features: 977
[LightGBM] [Info] Start training from score -1.098586
[LightGBM] [Info] Start training from score -1.098586
[LightGBM] [Info] Start training from score -1.098665


[I 2024-10-19 18:47:57,713] Trial 29 finished with value: 0.7430851063829788 and parameters: {'n_estimators': 220, 'learning_rate': 0.01760199127864501, 'max_depth': 10}. Best is trial 26 with value: 0.8059574468085107.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.806331 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 99989
[LightGBM] [Info] Number of data points in the train set: 37598, number of used features: 977
[LightGBM] [Info] Start training from score -1.098586
[LightGBM] [Info] Start training from score -1.098586
[LightGBM] [Info] Start training from score -1.098665
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] N

2024/10/19 18:50:27 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/19 18:50:44 INFO mlflow.tracking._tracking_service.client: 🏃 View run LightGBM_SMOTE_TFIDF_Trigrams at: https://dagshub.com/rajatchauhan99/Youtube-Comment-Analysis-Chrome-Plugin.mlflow/#/experiments/5/runs/7a391a4289e441be884aeaad2a1ab437.
2024/10/19 18:50:44 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/rajatchauhan99/Youtube-Comment-Analysis-Chrome-Plugin.mlflow/#/experiments/5.
